In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertConfig, DistilBertForTokenClassification

# Build an English pipeline
#stanza.download('en', package='mimic', processors={'ner': 'i2b2'}) # download English model
#nlp = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'}) # initialize English neural pipeline

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
token_df = pd.read_csv('//home/chudeo/project/33k_sentence.csv')

In [4]:
token_df.head()

,sentence_id,words,labels
0,0,Baseline,O
1,0,artifact,O
2,0,.,O
3,1,Probable,O
4,1,sinus,B


In [5]:
token_df.count()

sentence_id    625510
words          625262
labels         625510
dtype: int64

In [7]:
#checking for null values
token_df.isnull().sum()

sentence_id      0
words          248
labels           0
dtype: int64

In [8]:
data = token_df.fillna(method='ffill')
data.head()

/tmp/ipykernel_2374583/3831777956.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = token_df.fillna(method='ffill')


,sentence_id,words,labels
0,0,Baseline,O
1,0,artifact,O
2,0,.,O
3,1,Probable,O
4,1,sinus,B


In [9]:
# let's create a new column called "sentence" which groups the words by sentence
data['sentence'] = data[['sentence_id','words','labels']].groupby(['sentence_id'])['words'].transform(lambda x: ' '.join(x))
# let's also create a new column called "word_labels" which groups the tags by sentence
data['word_labels'] = data[['sentence_id','words','labels']].groupby(['sentence_id'])['labels'].transform(lambda x: ','.join(x))
data.head()

,sentence_id,words,labels,sentence,word_labels
0,0,Baseline,O,Baseline artifact .,"O,O,O"
1,0,artifact,O,Baseline artifact .,"O,O,O"
2,0,.,O,Baseline artifact .,"O,O,O"
3,1,Probable,O,Probable sinus tachycardia with atrial prematu...,"O,B,I,O,O,O,O,O"
4,1,sinus,B,Probable sinus tachycardia with atrial prematu...,"O,B,I,O,O,O,O,O"


In [10]:

label2id = {k: v for v, k in enumerate(data.labels.unique())}
id2label = {v: k for v, k in enumerate(data.labels.unique())}
label2id

{'O': 0, 'B': 1, 'I': 2}

In [11]:
data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

,sentence,word_labels
0,Baseline artifact .,"O,O,O"
1,Probable sinus tachycardia with atrial prematu...,"O,B,I,O,O,O,O,O"
2,Low limb lead voltage .,"O,O,O,O,O"
3,Leftward axis .,"O,O,O"
4,Late R wave progression .,"O,O,O,O,O"


In [12]:
len(data)


11262

In [13]:
data.iloc[1].sentence

'Probable sinus tachycardia with atrial premature beats .'

In [14]:
data.iloc[1].word_labels

'O,B,I,O,O,O,O,O'

#### **Keeping only Evidence sentence**

In [15]:
filtered_df = data[~data['word_labels'].apply(lambda x: all(label == 'O' for label in x.split(',')))]


In [16]:
len(filtered_df)

2189

In [17]:
data = filtered_df.reset_index(drop=True)

In [18]:
data.iloc[1].sentence

'Other ST - T wave abnormalities .'

In [19]:
data.iloc[1].word_labels

'O,B,I,I,I,I,O'

In [20]:
from transformers import BertTokenizer, BertForTokenClassification, AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [21]:
# Extract words and labels from DataFrame
split_data = {
    "words": [word for sent in data["sentence"].str.split() for word in sent],
    "labels": [label.split(',') for label in data["word_labels"]]
}

In [22]:
# Split data into sentences and labels
sentences = split_data["words"]
labels = split_data["labels"]

In [23]:
# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
tokenized_sentences = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [24]:
# Define classes
tag2idx = {"O": 0, "B": 1, "I": 2}
idx2tag = {idx: tag for tag, idx in tag2idx.items()}
num_classes = len(tag2idx)
MAX_LEN= 128
TRAIN_BATCH_SIZE =16
VALID_BATCH_SIZE = 8

In [25]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        tokenized_sentence.extend(tokenized_word)
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [26]:
class NERDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        sentence = self.data.sentence[index]
        word_labels = self.data.word_labels[index]
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)

        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"]
        labels.insert(0, "O")
        labels.insert(-1, "O")

        if len(tokenized_sentence) > self.max_len:
            tokenized_sentence = tokenized_sentence[:self.max_len]
            labels = labels[:self.max_len]
        else:
            tokenized_sentence = tokenized_sentence + ['[PAD]' for _ in range(self.max_len - len(tokenized_sentence))]
            labels = labels + ["O" for _ in range(self.max_len - len(labels))]

        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)
        label_ids = [label2id[label] for label in labels]

        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(attn_mask, dtype=torch.long),
            'labels': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len


In [27]:
# Assuming you have initialized tokenizer, label2id, and MAX_LEN earlier
train_size = 0.8
train_data = data.sample(frac=train_size, random_state=200)
test_data = data.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

train_dataset = NERDataset(train_data, tokenizer, MAX_LEN)
test_dataset = NERDataset(test_data, tokenizer, MAX_LEN)


FULL Dataset: (2189, 2)
TRAIN Dataset: (1751, 2)
TEST Dataset: (438, 2)


In [28]:
train_dataset[0]

{'input_ids': tensor([  101,  2381,  1997,  2556,  7355,  1024,  3770,  1042,  2007,  4187,
          2004,  1006,  1031,  1008,  1008,  3295,  1006,  4895,  1007, 11518,
          1008,  1008,  1033,  1014,  1012,  1021,  4642,  2475,  2011, 23746,
          2063,  1031,  1008,  1008, 17405,  2575,  1011,  2340,  1011,  2484,
          1008,  1008,  1033,  1007,  1010, 28697,  2497,  1055,  1013,  1052,
          3144,  5887,  2278,  2615,  1031,  1008,  1008, 17405,  2575,  1011,
          2340,  1011,  1023,  1008,  1008,  1033,  1010,  7020,  2015,  1055,
          1013,  1052,  4903,  2213,  1010,  2203, 24755, 17080,  7315,  1055,
          1013,  1052, 19842,  7192,  1010,  1998,  8777, 21908, 23760, 29048,
          2040,  7534,  2007,  2184,  2154,  2381,  1997,  4788,  5582,  1040,
          7274,  2361, 22084,  1012,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [29]:
train_dataset[0]['input_ids']

tensor([  101,  2381,  1997,  2556,  7355,  1024,  3770,  1042,  2007,  4187,
         2004,  1006,  1031,  1008,  1008,  3295,  1006,  4895,  1007, 11518,
         1008,  1008,  1033,  1014,  1012,  1021,  4642,  2475,  2011, 23746,
         2063,  1031,  1008,  1008, 17405,  2575,  1011,  2340,  1011,  2484,
         1008,  1008,  1033,  1007,  1010, 28697,  2497,  1055,  1013,  1052,
         3144,  5887,  2278,  2615,  1031,  1008,  1008, 17405,  2575,  1011,
         2340,  1011,  1023,  1008,  1008,  1033,  1010,  7020,  2015,  1055,
         1013,  1052,  4903,  2213,  1010,  2203, 24755, 17080,  7315,  1055,
         1013,  1052, 19842,  7192,  1010,  1998,  8777, 21908, 23760, 29048,
         2040,  7534,  2007,  2184,  2154,  2381,  1997,  4788,  5582,  1040,
         7274,  2361, 22084,  1012,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [30]:
# print the first 30 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(train_dataset[0]["input_ids"][:30]), train_dataset[0]["labels"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[CLS]       O
history     O
of          O
present     O
illness     O
:           O
80          O
f           O
with        O
critical    O
as          B
(           O
[           O
*           O
*           O
location    O
(           O
un          O
)           O
109         O
*           O
*           O
]           O
0           O
.           O
7           O
cm          O
##2         O
by          O
tt          O


In [31]:
# Define your DataLoader parameters
train_params = {'batch_size': TRAIN_BATCH_SIZE, 'shuffle': True, 'num_workers': 0}
test_params = {'batch_size': VALID_BATCH_SIZE, 'shuffle': False, 'num_workers': 0}

# Create your DataLoader
train_loader = DataLoader(train_dataset, **train_params)
test_loader = DataLoader(test_dataset, **test_params)

In [32]:

# Define model
model = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=num_classes)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=1e-05)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/chudeo/using_gpu/lib/python3.9/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [33]:
import torch
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

In [34]:
ids = train_dataset[0]["input_ids"].unsqueeze(0)
mask = train_dataset[0]["attention_mask"].unsqueeze(0)
targets = train_dataset[0]["labels"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(1.0376, device='cuda:0', grad_fn=<NllLossBackward0>)

In [35]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 3])

In [36]:
MAX_GRAD_NORM = 10

In [37]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(train_loader):

        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        targets = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [38]:
EPOCHS = 10
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 1.0132217407226562
Training loss per 100 training steps: 0.24518628470083276
Training loss epoch: 0.23414812697605653
Training accuracy epoch: 0.8400966460319026
Training epoch: 2
Training loss per 100 training steps: 0.11454218626022339
Training loss per 100 training steps: 0.10368164141874502
Training loss epoch: 0.1030302570624785
Training accuracy epoch: 0.8729934605429258
Training epoch: 3
Training loss per 100 training steps: 0.08714299649000168
Training loss per 100 training steps: 0.08194089415344862
Training loss epoch: 0.0809668557210402
Training accuracy epoch: 0.8984413580162084
Training epoch: 4
Training loss per 100 training steps: 0.06641068309545517
Training loss per 100 training steps: 0.06844351883276854
Training loss epoch: 0.06794511513276534
Training accuracy epoch: 0.9160242833469302
Training epoch: 5
Training loss per 100 training steps: 0.09174048155546188
Training loss per 100 training steps: 0.05767636133

In [39]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            targets = batch['labels'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [40]:
labels, predictions = valid(model, test_loader)

Validation loss per 100 evaluation steps: 0.04396697133779526
Validation Loss: 0.07478406888178804
Validation Accuracy: 0.9227051630707879


In [41]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

              precision    recall  f1-score   support

           _       0.66      0.68      0.67      1526

   micro avg       0.66      0.68      0.67      1526
   macro avg       0.66      0.68      0.67      1526
weighted avg       0.66      0.68      0.67      1526



In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Compute accuracy
accuracy = accuracy_score(labels, predictions)

# Compute precision
precision = precision_score(labels, predictions, average='weighted')

# Compute recall
recall = recall_score(labels, predictions, average='weighted')

# Compute F1-score
f1 = f1_score(labels, predictions, average='weighted')

# Classification report
report = classification_report(labels, predictions)

# Confusion matrix
conf_matrix = confusion_matrix(labels, predictions)

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-Score: {:.4f}".format(f1))
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)


Accuracy: 0.9282
Precision: 0.9264
Recall: 0.9282
F1-Score: 0.9271

Classification Report:
               precision    recall  f1-score   support

           B       0.74      0.72      0.73      1515
           I       0.74      0.67      0.71      1168
           O       0.96      0.97      0.96     14899

    accuracy                           0.93     17582
   macro avg       0.81      0.79      0.80     17582
weighted avg       0.93      0.93      0.93     17582


Confusion Matrix:
 [[ 1094    74   347]
 [  121   788   259]
 [  263   199 14437]]


In [43]:
from nervaluate import Evaluator

In [44]:
# Replace the existing evaluation code
labels, predictions = valid(model, test_loader)

# Use nervaluate to evaluate the model
evaluator = Evaluator(labels, predictions, true_label_scheme='IOB', pred_label_scheme='IOB')
results = evaluator.evaluate()

# Print the evaluation results
print(results)

Validation loss per 100 evaluation steps: 0.04396697133779526
Validation Loss: 0.07478406888178804
Validation Accuracy: 0.9227051630707879


TypeError: __init__() got an unexpected keyword argument 'true_label_scheme'